# Example of generating summaries for a 10K
Source: https://investors.nike.com/investors/news-events-and-reports/

### Before running the code

You will need to have the following packages installed:
```
pip3 install langchain pandas pypdf
```

Also, make sure you have a .env file with your OpenAI API key in the root directory of this project.
```
OPENAI_API_KEY=YOUR_API_KEY
```

### Load packages

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from uniflow.client import Client
from uniflow.config import OpenAIJsonConfig
from langchain.document_loaders import PyPDFLoader
from uniflow.schema import Context, GuidedPrompt

load_dotenv()


/Users/joseortiz/anaconda3/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Prepare the input data

In [3]:
pdf_file = "nike-10k-2023.pdf"

##### Set current directory and input data directory.

In [4]:
dir_cur = os.getcwd()
input_file = os.path.join(f"{dir_cur}/data/raw_input/", pdf_file)

##### Load and split the pdf

In [5]:
loader = PyPDFLoader(input_file)
pages = loader.load_and_split()
page_contents = [page.page_content for page in pages]

In [6]:
guided_prompt = GuidedPrompt(
    instruction="Generate a one sentence summary based on the last context below. Follow the format of the examples below to include context and summary in the response",
    examples=[
        Context(
            context="When you're operating on the maker's schedule, meetings are a disaster. A single meeting can blow a whole afternoon, by breaking it into two pieces each too small to do anything hard in. Plus you have to remember to go to the meeting. That's no problem for someone on the manager's schedule. There's always something coming on the next hour; the only question is what. But when someone on the maker's schedule has a meeting, they have to think about it.",
            summary="Meetings disrupt the productivity of those following a maker's schedule, dividing their time into impractical segments, while those on a manager's schedule are accustomed to a continuous flow of tasks.",
        ),
    ],
)
data = [ Context(context=p[:500], summary="") for p in page_contents[6:16] if len(p) > 200 ]

In [7]:
data

[Context(context='We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth \nfiscal quarters have slightly exceeded those in the second and third ', summary=''),
 Context(context='INTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 60% \nand 61% for fiscal 2022 and fiscal 2021, respectively. We sell our products to retail accounts through our own NIKE Direct \noperations and through a mix of independent distributors, licensees and sales representatives around the world. W e sell to \nthousands of retail accounts and ship produc

### Run the model

In [8]:
config = OpenAIJsonConfig(guided_prompt_template=guided_prompt)
client = Client(config)

In [9]:
output = client.run(data)

100%|██████████| 10/10 [00:24<00:00,  2.49s/it]


In [10]:
output

[{'output': [{'response': [{'context': 'We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth \nfiscal quarters have slightly exceeded those in the second and third',
      'summary': 'Revenue fluctuations are experienced throughout the year, with revenues in the first and fourth fiscal quarters slightly surpassing those in the second and third.'}],
    'error': 'No errors.'}],
  'root': <uniflow.node.node.Node at 0x1573304c0>},
 {'output': [{'response': [{'context': 'INTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 60% \

### Process the output

In [11]:
# Extracting context, question, and answer into a DataFrame
contexts = []
summaries = []

for item in output:
    for i in item.get('output', []):
        for response in i.get('response', []):
            if any(key not in response for key in ['context', 'summary']):
                print("Missing context or summary in response:", response)
                continue
            contexts.append(response['context'])
            summaries.append(response['summary'])

# Set display options
pd.set_option('display.max_colwidth', None)  # or use a specific width like 50
pd.set_option('display.width', 1000)

df = pd.DataFrame({
    'Context': contexts,
    'Summaries': summaries,
})

df

,Context,Summaries
0,"We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth \nfiscal quarters have slightly exceeded those in the second and third","Revenue fluctuations are experienced throughout the year, with revenues in the first and fourth fiscal quarters slightly surpassing those in the second and third."
1,"INTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 60% \nand 61% for fiscal 2022 and fiscal 2021, respectively. We sell our products to retail accounts through our own NIKE Direct \noperations and through a mix of independent distributors, licensees and sales representatives around the world. W e sell to \nthousands of retail accounts and ship products from 67 distribution centers outside of the United States.","NIKE's non-U.S sales accounted for 57% of total revenues in fiscal 2023, demonstrating a slight decrease compared to previous fiscal years, and the company operates worldwide through various sales channels."
2,"In fiscal 2023, Vietnam, Indonesia, and China constituted majority of NIKE Brand footwear production with 50%, 27%, and 18% respectively, along with four contract manufacturers responsible for over 10% each and operating 291 finished goods apparel factories in 31 countries.","NIKE's footwear production locations for fiscal 2023 were primarily in Vietnam, Indonesia, and China, with over half of production managed by four contract manufacturers operating across 31 countries."
3,"of total NIKE Brand apparel, respectively. For fiscal 2023, one apparel contract manufacturer accounted for more than 10% of \napparel production, and the top five contract manufacturers in the aggregate accounted for approximately 52% of NIKE Brand \napparel production.\nNIKE's contract manufacturers buy raw materials for the manufacturing of our footwear, apparel and equipment products. Most \nraw materials are available and purchased by those contract manufacturers in the countries where manufact","NIKE relies heavily on contract manufacturers for the production of its apparel, with the top five manufacturers accounting for over half of its apparel production."
4,"We monitor protectionist trends and developments throughout the world that may materially impact our industry, and we engage in administrative and judicial processes to mitigate trade restrictions. We are actively monitoring actions that may result in additional anti-dumping measures and could affect our industry. We are also monitoring for and advocating against other impediments that may limit or delay customs clearance for imports of footwear, apparel and equipment. NIKE also advocates f","NIKE monitors protectionist trends globally and engages in administrative and judicial processes to mitigate trade restrictions that could impact the import of footwear, apparel, and equipment."
5,"Our international operations are also subject to compliance with the U.S . Foreign Corrupt Practices Act (the ""FCPA""), and other \nanti-bribery laws applicable to our operations. We source a significant portion of our products from, and have important consumer \nmarkets, outside of the United States. We have an ethics and compliance programto address compliance with the FCPA and \nsimilar laws by us, our employees, agents, suppliers and other partners. Refer to Item 1A. Risk Factors for additiona","International operations are subject to adherence with the FCPA and anti-bribery laws, necessitating the implementation of an ethics and compliance program to address compliance guidelines for all relevant parties."
6,"Our suc

In [14]:
output_dir = 'data/output'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df.to_csv(f"{output_dir}/Nike_10k_Summaries.csv", index=False)